This example is based on the `01_match_lengths.ipynb` example with the additional change that when no more tokens can be matched the remaining unmatched tokens are moved to new places to indicate that they could not be matched up.

In [ ]:
from typing import Optional, Union
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization

from petritype.core.executable_graph_components import (
    ListPlaceNode, FunctionTransitionNode, ArgumentEdgeToTransition, ReturnedEdgeFromTransition,
    ExecutableGraphOperations
)
from petritype.core.rustworkx_graph import RustworkxGraph
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from IPython.display import display, clear_output

In [ ]:
# NOTE: In practice you would probably want to define pydantic models for the input and output types.
def match_one_string_to_one_length(
    strings: list[str], lengths: list[int]
) -> tuple[
    list[str], list[int], Optional[tuple[str, int]], Optional[list[str]], Optional[list[int]]
]:
    """An impure function that pulls matching pairs out of the input lists."""
    matched_pair = None
    for s in strings:
        for l in lengths:
            if len(s) == l:
                matched_pair = (s, l)
                # Remove matched items from input lists
                strings.remove(s)
                lengths.remove(l)
                return strings, lengths, matched_pair, None, None
    # No match found
    return None, None, None, strings, lengths

def distribute_result_tokens(
    result: tuple[
        list[str], list[int], Optional[tuple[str, int]], Optional[list[str]], Optional[list[int]]
    ]
) -> dict[str, Union[list[str], list[int], tuple[str, int]]]:
    strings, lengths, matched_pair, unmatched_strings, unmatched_lengths = result
    print("Distributing result tokens:")
    print(f"  Some Strings: {strings}")
    print(f"  Some Lengths: {lengths}")
    print(f"  Matched Pair: {matched_pair}")
    print(f"  Unmatched Strings: {unmatched_strings}")
    print(f"  Unmatched Lengths: {unmatched_lengths}")
    non_none_results = {}
    if strings is not None:
        non_none_results["Some Strings"] = strings
    if lengths is not None:
        non_none_results["Some Lengths"] = lengths
    if matched_pair is not None:
        non_none_results["Matched Pair"] = matched_pair
    if unmatched_strings is not None:
        non_none_results["Unmatched Strings"] = unmatched_strings
    if unmatched_lengths is not None:
        non_none_results["Unmatched Lengths"] = unmatched_lengths
    return non_none_results

In [ ]:
executable_graph_nodes_and_edges = [
    ListPlaceNode("Some Strings", str, ["a", "ab", "abc", "abcd"]),
    ListPlaceNode("Some Lengths", int, [2, 3, 4, 5, 99]),
    
    ArgumentEdgeToTransition("Some Strings", "Match Lengths", "strings"),
    ArgumentEdgeToTransition("Some Lengths", "Match Lengths", "lengths"),
    FunctionTransitionNode(
        "Match Lengths",
        match_one_string_to_one_length,
        output_distribution_function=distribute_result_tokens,
    ),
    ReturnedEdgeFromTransition("Match Lengths", "Some Strings"),
    ReturnedEdgeFromTransition("Match Lengths", "Some Lengths"),
    ReturnedEdgeFromTransition("Match Lengths", "Matched Pair"),
    ReturnedEdgeFromTransition("Match Lengths", "Unmatched Strings"),
    ReturnedEdgeFromTransition("Match Lengths", "Unmatched Lengths"),

    ListPlaceNode("Matched Pair", Optional[tuple[str, int]], []),
    ListPlaceNode("Unmatched Strings", Optional[str], []),
    ListPlaceNode("Unmatched Lengths", Optional[int], []),
]

In [ ]:
executable_graph = ExecutableGraphOperations.construct_graph(executable_graph_nodes_and_edges)
executable_pydigraph = RustworkxGraph.from_executable_graph(executable_graph)
SimpleGraphvizVisualization.graph(executable_pydigraph)

In [ ]:
async for step, diagram, transitions_fired in SimpleGraphvizVisualization.animate_execution_generator(
    executable_graph=executable_graph,
    executable_pydigraph=executable_pydigraph,
):
    clear_output(wait=True)
    print(f"Step {step}")
    display(diagram)
    print(f"Transitions fired: {transitions_fired}")
    if not transitions_fired:
        print("No more transitions to fire. Execution complete.")
        break
    time.sleep(1.0)
    plt.close()